Set the `sample_area` boundaries and the number of samples (`total_dots`) to take

In [ ]:
"""mc_circle_prng.ipynb"""

# Cell 01

from matplotlib.patches import Rectangle

# ((x, y) anchor point, width, height)
bbox = Rectangle((-1, -1), 2, 2).get_bbox()
print(bbox)

total_dots = 320 * 320  # 102_400
print(f"{total_dots = :,}")

Set the numpy PRNG seed to 2020 and take $n$ random samples of 2D Cartesian points $(x,y)$
1. Use the built-in Python `uniform` distribution which returns a random float [0,1)
2. Subtract that float from 1, so the interval flips to (0,1] ensuring any points\
   on the perimeter will now contribute to the area
3. Scale the result so it now falls in sample area

In [ ]:
# Cell 02

import numpy as np
import pandas as pd

rng = np.random.default_rng(seed=2020)

x = (1 - rng.random(total_dots)) * bbox.width + bbox.x0
y = (1 - rng.random(total_dots)) * bbox.height + bbox.y0

pd.DataFrame({"x": x[:5], "y": y[:5]})

Create an array $d$ that contains the distance from the origin $(0,0)$ for every point $(x,y)$\
Leverage the fact the exponentiation and addition operators are "vector aware"

In [ ]:
# Cell 03

d = np.sqrt(x**2 + y**2)

pd.DataFrame({"x": x[:5], "y": y[:5], "d": d[:5]})

Create arrays of $(x,y)$ coordinates that are "on or inside" vs. "outside" the circle\
using the Pythagorean distance $d$\
Leverage the ability to `filter` numpy arrays using a conditional expression

In [ ]:
# Cell 04

x_in = x[d <= 1.0]  # On or inside the circle
y_in = y[d <= 1.0]
x_out = x[d > 1.0]  # Outside the circle
y_out = y[d > 1.0]

pd.DataFrame(
    {"x_in": x_in[:5], "y_in": y_in[:5], "x_out": x_out[:5], "y_out": y_out[:5]}
)

Display the scatter plot of the Monte Carlo estimation

In [ ]:
# Cell 05

import matplotlib.pyplot as plt

plt.figure(figsize=(10, 10))
plt.scatter(x_in, y_in, color="red", marker=".", s=0.5)
plt.scatter(x_out, y_out, color="blue", marker=".", s=0.5)
plt.gca().set_aspect("equal")
plt.show()

Calculate the absolute percent error in the area estimation
1. The actual/expected area of a unit circle is exactly $\pi$
2. The observed/estimated area using the Monte Carlo formulation $=4\times\dfrac{dots_{\ inside}}{dots_{\ total}}$


In [ ]:
# Cell 06

act = np.pi
est = (bbox.width * bbox.height) * np.count_nonzero(d <= 1.0) / total_dots
err = np.abs((est - act) / act)

print(f"dots = {total_dots:,}")
print(f"act = {act:.6f}")
print(f"est = {est:.6f}")
print(f"err = {err:.5%}")